In [16]:
from pathlib import Path
import torch
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from torchvision.models import resnet50, ResNet50_Weights
from tqdm import tqdm

data_path = Path("/media/alberto/Elements/Datasets/COCO/val2017_cropped")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
def train_loop(model, loader, criterion, optimizer, train_steps):
    model.train()
    train_iter = iter(loader)

    for step in range(train_steps):
        try:
            inputs, targets = next(train_iter)
        except StopIteration:
            train_iter = iter(loader)
            inputs, targets = next(train_iter)

        optimizer.zero_grad()
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        if step % 100 == 0:
            print(f"Step [{step}/{train_steps}]: Loss {loss.item()}")


In [19]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])
transform_test = transforms.Compose([
    transforms.ToTensor()
])
dataset = ImageFolder(
    root = data_path,
    transform = transform_train,
    target_transform = None
)
loader = DataLoader(
    dataset,
    batch_size=32,
    shuffle=True
)



model = resnet50(ResNet50_Weights.DEFAULT)
# model.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, 2)
model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
train_loop(model, loader, criterion, optimizer, train_steps=1000)


/home/alberto/Repos/env_cv/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


Step [0/1000]: Loss 0.6700745820999146
